
# Tuberculosis X-ray Segmentation using Hugging Face

### This notebook implements a complete pipeline for segmenting tuberculosis in X-ray images using the SegFormer model from Hugging Face Transformers.


In [4]:
# Import the necessary libraries
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import SegformerForSemanticSegmentation, SegformerConfig
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm



/Users/danielharrod/anaconda3/envs/TB_Image/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Here we are making sure that the data in this cases images, all have to have the same images. 
class ResizeAndPad:
    def __init__(self, target_size=(128, 128)):
        self.target_size = target_size
        
    def __call__(self, image, mask):
        # Resize maintaining aspect ratio
        aspect_ratio = image.size[0] / image.size[1]
        if aspect_ratio > 1:
            new_width = self.target_size[0]
            new_height = int(new_width / aspect_ratio)
        else:
            new_height = self.target_size[1]
            new_width = int(new_height * aspect_ratio)
            
        image = image.resize((new_width, new_height), Image.Resampling.BILINEAR)
        mask = mask.resize((new_width, new_height), Image.Resampling.NEAREST)
        
        # Create new images with padding
        padded_image = Image.new('RGB', self.target_size, (0, 0, 0))
        padded_mask = Image.new('L', self.target_size, 0)
        
        # Calculate padding
        left = (self.target_size[0] - new_width) // 2
        top = (self.target_size[1] - new_height) // 2
        
        # Paste the resized images
        padded_image.paste(image, (left, top))
        padded_mask.paste(mask, (left, top))
        
        return padded_image, padded_mask

In [6]:
class TBDataset(Dataset):
    def __init__(self, image_dir, mask_dir, metadata_path, target_size=(128, 128)):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.metadata = pd.read_csv(metadata_path)
        self.image_files = sorted(os.listdir(image_dir))
        self.target_size = target_size
        
        # Define transforms
        self.resize_and_pad = ResizeAndPad(target_size)
        self.image_transforms = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Load image and mask
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        mask_path = os.path.join(self.mask_dir, self.image_files[idx].replace('.jpg', '_mask.png'))
        
        # Read image and mask
        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')
        
        # Apply resize and padding
        image, mask = self.resize_and_pad(image, mask)
        
        # Convert to tensors and normalize image
        image = self.image_transforms(image)
        mask = transforms.ToTensor()(mask)
        
        return image, mask

In [7]:
# Set up the data loading pipeline with proper transformations and splits.
def create_data_loaders(image_dir, mask_dir, metadata_path, batch_size=8):
    
    # Create dataset
    dataset = TBDataset(image_dir, mask_dir, metadata_path)
    
    # Split dataset
    train_idx, val_idx = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
    train_dataset = torch.utils.data.Subset(dataset, train_idx)
    val_dataset = torch.utils.data.Subset(dataset, val_idx)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    return train_loader, val_loader

In [8]:
def train_model(model, train_loader, val_loader, num_epochs=10, device='cuda'):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    criterion = torch.nn.CrossEntropyLoss()
    
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0
        for images, masks in tqdm(train_loader):
            # Debugging shapes
            tqdm.write(f"Images shape: {images.shape}")
            tqdm.write(f"Masks shape: {masks.shape}")
            
            images = images.to(device)
            # Add a channel dimension to masks, resize, and ensure correct shape
            if len(masks.shape) == 1:  # If shape is [B, H, W], add channel dimension
                masks = masks.unsqueeze(1)
            masks_resized = F.interpolate(masks.float(), size=(32, 32), mode='nearest').squeeze(1).long().to(device)  
            
            # Now we can print the resized shape
            tqdm.write(f"Resized masks shape: {masks_resized.shape}")

            outputs = model(images).logits  # Ensure model outputs logits
            loss = criterion(outputs, masks_resized)  # Use masks_resized here
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(device)
                # Add a channel dimension to masks, resize, and ensure correct shape
                if len(masks.shape) == 1: 
                    masks = masks.unsqueeze(1)
                masks_resized = F.interpolate(masks.float(), size=(32, 32), mode='nearest').squeeze(1).long().to(device)  

                outputs = model(images).logits
                loss = criterion(outputs, masks_resized)  # Use masks_resized here
                val_loss += loss.item()

        # Print metrics for the epoch
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        # Save the model if validation loss improves
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print("Saved Best Model!")

            
    


In [9]:
# Set up the complete pipeline and run the training.
def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Define paths
    image_dir = './datasets/image'
    mask_dir = './datasets/mask'
    metadata_path = './datasets/MetaData.csv'

    # define the hyperparameters
    batch_size=8 
    target_size=(128, 128)
    
    # Create data loaders
    print("Creating data loaders...")
    train_loader, val_loader = create_data_loaders(image_dir, mask_dir, metadata_path)    
    
    # Initialize model
    print("Initializing model...")
    config = SegformerConfig.from_pretrained(
        "nvidia/mit-b0",
        num_labels=2,
        id2label={0: "background", 1: "tuberculosis"},
        label2id={"background": 0, "tuberculosis": 1}
    )
    model = SegformerForSemanticSegmentation.from_pretrained(
        "nvidia/mit-b0",
        config=config
    )
    
    # Train model
    print("Starting training...")
    train_model(model, train_loader, val_loader, device=device)
    print("Training completed!")

In [10]:
if __name__ == "__main__":
    main()




def visualize_prediction(model, image_tensor, device):
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.unsqueeze(0).to(device)
        output = model(image_tensor).logits
        prediction = torch.sigmoid(output) > 0.5
        
    # Convert to numpy for visualization
    prediction = prediction.cpu().numpy()[0, 0]
    
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image_tensor[0].cpu().permute(1, 2, 0))
    plt.title('Original Image')
    plt.subplot(1, 2, 2)
    plt.imshow(prediction, cmap='gray')
    plt.title('Segmentation Mask')
    plt.show()

Using device: cpu
Creating data loaders...
Initializing model...


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting training...


  0%|          | 0/71 [00:00<?, ?it/s]

Images shape: torch.Size([8, 3, 128, 128])
Masks shape: torch.Size([8, 1, 128, 128])
Resized masks shape: torch.Size([8, 32, 32])


...

Images shape: torch.Size([8, 3, 128, 128])
Masks shape: torch.Size([8, 1, 128, 128])
Resized masks shape: torch.Size([8, 32, 32])


 99%|█████████▊| 70/71 [01:27<00:01,  1.34s/it]

Images shape: torch.Size([3, 3, 128, 128])
Masks shape: torch.Size([3, 1, 128, 128])
Resized masks shape: torch.Size([3, 32, 32])


100%|██████████| 71/71 [01:27<00:00,  1.23s/it]


Epoch 10/10, Train Loss: 0.0832, Val Loss: 0.0877
Saved Best Model!
Training completed!
